In [1]:
import json 

json_obj1 = {"title": "Sample Document",
    "data": "This is a sample document.",
    "tags": ["sample"]}
json_obj2 = {"title": "Test Document",
    "data": "This is a  test document.",
    "tags": ["test","second test"]}

json_rows = [json_obj1,json_obj2] 

with open('sample_data.jsonl', 'w') as file:
    for row in json_rows:
        file.write(json.dumps(row) + '\n')

In [2]:
text = """
Marie Curie, born in 1867, was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity.
She was the first woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields.
Her husband, Pierre Curie, was a co-winner of her first Nobel Prize, making them the first-ever married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes.
She was, in 1906, the first woman to become a professor at the University of Paris.
"""

In [3]:
file_path = 'graph.json'

# Open and load the JSON file
with open(file_path, 'r') as file:
    data = json.load(file)

json_data = {"data" : f'{data}'} 

with open('new_sample_data.jsonl','w') as f:
    f.write(json.dumps(json_data) + '\n')

In [4]:
import pathway as pw

from pathway.xpacks.llm.vector_store import VectorStoreServer
# from pathway.xpacks.llm.splitters import TokenCountSplitter

/home/vectornd/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [5]:
class InputSchema(pw.Schema):
  title : str 
  data : str 
  tags : list 
  
table_json = pw.io.jsonlines.read("sample_data.jsonl", schema=InputSchema, mode="static", with_metadata=True) 

In [6]:
class InSchema(pw.Schema):
  data : pw.Json
  
table_kg = pw.io.jsonlines.read("new_sample_data.jsonl", schema=InSchema, mode="static", with_metadata=True) 

In [7]:
table_kg

[2024-11-03T22:25:29]:INFO:Preparing Pathway computation
[2024-11-03T22:25:29]:INFO:FilesystemReader-0: 0 entries (1 minibatch(es)) have been sent to the engine
[2024-11-03T22:25:29]:INFO:FilesystemReader-0: 1 entries (2 minibatch(es)) have been sent to the engine
[2024-11-03T22:25:29]:WARNING:FilesystemReader-0: Closing the data source


BokehModel(combine_events=True, render_bundle={'docs_json': {'505ed594-d6b2-4ee1-96f1-737382b5b13e': {'version…

In [8]:
from pathway.xpacks.llm import embedders
from pathway.xpacks.llm.splitters import TokenCountSplitter
from pathway.xpacks.llm.vector_store import VectorStoreClient, VectorStoreServer

PATHWAY_PORT = 8764

text_splitter = TokenCountSplitter()
embedder = embedders.SentenceTransformerEmbedder(model="all-MiniLM-L6-v2")

vector_server = VectorStoreServer(
    table_kg,
    embedder=embedder
)
vector_server.run_server(host="127.0.0.1", port=PATHWAY_PORT, threaded=True, with_cache=False)
# time.sleep(30)  # Workaround for Colab - messages from threads are not visible unless a cell is running


/home/vectornd/anaconda3/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
[2024-11-03T22:25:31]:INFO:Load pretrained SentenceTransformer: all-MiniLM-L6-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

<Thread(VectorStoreServer, started 136723715589824)>

In [9]:
embedder

In [11]:
client = VectorStoreClient(
    host="127.0.0.1",
    port=PATHWAY_PORT,
)

In [12]:
query = "What is a node?"
docs = client(query)
docs

[2024-11-03T22:25:55]:INFO:{"_type": "request_payload", "session_id": "uuid-be47ee9b-f51b-4d72-8f2b-2ec39c75b70f", "payload": {"query": "What is a node?", "k": 3}}
[2024-11-03T22:25:55]:INFO:PythonReader-1: 1 entries (121 minibatch(es)) have been sent to the engine


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[2024-11-03T22:25:55]:INFO:{"_type": "http_access", "method": "POST", "scheme": "http", "scheme_with_forwarded": "http", "host": "127.0.0.1:8764", "route": "/v1/retrieve", "content_type": "application/json", "user_agent": "python-requests/2.32.3", "unix_timestamp": 1730652955, "remote": "127.0.0.1", "session_id": "uuid-be47ee9b-f51b-4d72-8f2b-2ec39c75b70f", "headers": [{"header": "Host", "value": "127.0.0.1:8764"}, {"header": "User-Agent", "value": "python-requests/2.32.3"}, {"header": "Accept-Encoding", "value": "gzip, deflate, br, zstd"}, {"header": "Accept", "value": "*/*"}, {"header": "Connection", "value": "keep-alive"}, {"header": "Content-Type", "value": "application/json"}, {"header": "Content-Length", "value": "36"}], "status": 200, "time_elapsed": "0.131"}
[2024-11-03T22:25:55]:INFO:127.0.0.1 [03/Nov/2024:22:25:55 +0530] "POST /v1/retrieve HTTP/1.1" 200 4220 "-" "python-requests/2.32.3"


[{'dist': 0.7913080453872681,
  'metadata': {'created_at': 1730650096,
   'modified_at': 1730652925,
   'owner': 'vectornd',
   'path': 'new_sample_data.jsonl',
   'seen_at': 1730652949},
  'text': '{\'directed\': False, \'multigraph\': False, \'graph\': {}, \'nodes\': [{\'type\': \'PERSON\', \'description\': [\'Richard McClintock is a Latin professor at Hampden-Sydney College in Virginia\'], \'text_unit_ids\': [\'d3f79191-3913-4a30-b5d1-8a45f37f0561\'], \'degree\': 2, \'human_readable_id\': 0, \'id\': \'RICHARD MCCLINTOCK\'}, {\'type\': \'ORGANIZATION\', \'description\': [\'Hampden-Sydney College is an institution in Virginia\'], \'text_unit_ids\': [\'d3f79191-3913-4a30-b5d1-8a45f37f0561\'], \'degree\': 2, \'human_readable_id\': 1, \'id\': \'HAMPDEN-SYDNEY COLLEGE\'}, {\'type\': \'GEO\', \'description\': [\'Virginia is a state where Hampden-Sydney College is located\'], \'text_unit_ids\': [\'d3f79191-3913-4a30-b5d1-8a45f37f0561\'], \'degree\': 1, \'human_readable_id\': 2, \'id\': \'VI